# Web Assessment
- Author: Aidan Marlin
- Date: 20220623
- Version: 1.4
- Description: Jupyter Notebook to automate the execution of web scans

In [ ]:
target = "www.bing.com"
top_target = "bing.com"
ffuf_subdomain_target = "FUZZ.bing.com"
port = 443
### Set to 0 if scanning a subdomain
subdomain_recon = 1

###
### No need to modify from here
###
import socket
import os

os.environ["PATH"] = os.environ["PATH"] + ":/run/current-system/sw/bin"

target_ip = socket.gethostbyname(target) 
ffuf_subdomain_target_https = "https://" + ffuf_subdomain_target
ffuf_subdomain_target_http = "http://" + ffuf_subdomain_target
ffuf_directory_brute = target + ":" + str(port) + "/FUZZ"
http_target = "http://" + target
https_target = "https://" + target

!mkdir -p tools/amass
!mkdir -p tools/ctfr
!mkdir -p tools/ffuf_subdomain
!mkdir -p tools/ffuf_directory
!mkdir -p tools/knockpy
!mkdir -p tools/nettacker
!mkdir -p tools/nikto
!mkdir -p tools/photon
!mkdir -p tools/puppeteer
!mkdir -p tools/subfinder
!mkdir -p tools/sn1per
!mkdir -p tools/testssl
!mkdir -p tools/theharvester

def read_file(location):
    try:
        with open(location) as f:
            text = f.read()
            return text
    except:
        return False


# Target

In [ ]:
print(target)
print(target_ip)


## Whois

In [ ]:
!dig {target}


# Pentester Details

## IP Address

In [ ]:
!curl https://ifconfig.me/


## Start Time

In [ ]:
!date


## Interface Info

In [ ]:
!ip a


# Scans

## ctfr

In [ ]:
!time docker run --rm -v "`pwd`/tools/ctfr/:/mnt" unapibageek/ctfr -d $top_target -o /mnt/results.txt


## Corsy

In [ ]:
!time python "/home/user/git/pentest-tools/Corsy/corsy.py" -u {https_target}:{port}


## waybackpy

In [ ]:
!time docker run -it --rm secsi/waybackpy --url {target} --oldest


## getjs

In [ ]:
!time docker run -it --rm secsi/getjs --url {https_target}


## httpx

In [ ]:
!time echo {target} | httpx -silent -status-code -title -tech-detect -ip -asn


## dnstwist

In [ ]:
!time docker run --rm elceef/dnstwist {target}


## Gospider

In [ ]:
!time docker run -it --rm secsi/gospider -s {https_target}


## CloudFail

In [ ]:
!time docker run -it --rm secsi/cloudfail -u -t {target}


## CMSeek

In [ ]:
!time docker run -it --rm secsi/cmseek --batch -u {https_target}


## WhatWeb

In [ ]:
!time docker run --rm guidelacour/whatweb ./whatweb {https_target}:{port}


## hakrawler

In [ ]:
!time echo {https_target} | hakrawler | sort | uniq


## Subfinder

In [ ]:
%%bash -s "$subdomain_recon" "$top_target"
if [[ "$1" == "1" ]]; then
    # Scan
    time subfinder -d $2 -silent -active -oI -oJ -o tools/subfinder/results.txt 2>&1 >/dev/null
    # Output
    cat tools/subfinder/results.txt | jq -r '"Domain:  \(.host)\nIP addr: \(.ip)\n"'
fi


## Backup File Artifacts Checker

In [ ]:
!time docker run -it --rm secsi/bfac --no-text --url {https_target}


## Screenshot

In [ ]:
import shutil
from pathlib import Path
home = str(Path.home())

os.system("docker run -u root --shm-size 1G --rm -v \"" + os.getcwd() + "/tools/puppeteer:/screenshots\" alekzonder/puppeteer:latest screenshot " + https_target + ":" + str(port) + " 1024x768")
if not os.path.isfile("tools/puppeteer/screenshot_1024_768.png"):
    shutil.copyfile(home + "/git/nixos/resources/404.png", "tools/puppeteer/screenshot_1024_768.png")


![Target screenshot](tools/puppeteer/screenshot_1024_768.png)


## Email Enumeration

In [ ]:
!time docker run --rm infoga --domain $top_target --source all --breach -v 2


## amass

In [ ]:
!time amass enum -silent -nocolor -active -json tools/amass/results.txt -d {top_target} -p {port}


In [ ]:
!cat tools/amass/results.txt | jq -r '"##### \(.name)\nIP addr: \(.addresses|.[].ip)\nCIDR: \(.addresses|.[].cidr)\nASN: \(.addresses|.[].asn)\nDesc: \(.addresses|.[].desc)\n"'


## theHarvester

In [ ]:
!time docker run --rm -v "`pwd`/tools/theharvester:/mnt" simonthomas/theharvester:latest theharvester -d $top_target -b all -f /mnt/results.html


## ffuf (subdomains)

In [ ]:
%%bash -s "$subdomain_recon" "$ffuf_subdomain_target"

if [[ "$1" == "1" ]]; then
    echo https://$2
    # Scan
    time ffuf -s -of all -w /home/user/git/wordlists/SecLists/Discovery/DNS/subdomains-top1million-5000.txt -u https://$2 -od tools/ffuf_subdomain
    echo "`find tools/ffuf_subdomain -type f | wc -l` responses worth investigating.."
else
    echo "Subdomain scans have been disabled."
fi


## Knockpy

In [ ]:
!time docker run -it --rm -v "`pwd`/tools/knockpy:/mnt/" secsi/knockpy -o /mnt {top_target} 2>&1 > /dev/null


In [ ]:
!find tools/knockpy -type f -exec cat {} \;


## photon

In [ ]:
!time photon --keys -u {https_target} -l 3 -t 100 --wayback -o tools/photon/{target}


### files.txt

In [ ]:
print(read_file("tools/photon/" + target + "/files.txt"))


### fuzzable.txt

In [ ]:
print(read_file("tools/photon/" + target + "/fuzzable.txt"))


### intel.txt

In [ ]:
print(read_file("tools/photon/" + target + "/intel.txt"))


### scripts.txt

In [ ]:
print(read_file("tools/photon/" + target + "/scripts.txt"))


### external.txt

In [ ]:
!time cat tools/photon/{target}/external.txt | httpx -silent -status-code -location -title -tech-detect -cl -ct -fc 401,403,404


### internal.txt

In [ ]:
!time cat tools/photon/{target}/internal.txt | httpx -silent -status-code -location -title -tech-detect -cl -ct -fc 401,403,404


## is-website-vulnerable

In [ ]:
!time docker run --rm lirantal/is-website-vulnerable:latest {https_target}


## dirsearch

In [ ]:
!time docker run -it --rm secsi/dirsearch -q -x 301 -u {https_target}


## dvcs-ripper

In [ ]:
!time docker run -it --rm secsi/dvcs-ripper ./rip-svn.pl -u {https_target}


In [ ]:
!time docker run -it --rm secsi/dvcs-ripper ./rip-cvs.pl -u {https_target}


In [ ]:
!time docker run -it --rm secsi/dvcs-ripper ./rip-bzr.pl -u {https_target}


In [ ]:
!time docker run -it --rm secsi/dvcs-ripper ./rip-hg.pl -u {https_target}


In [ ]:
!time docker run -it --rm secsi/dvcs-ripper ./rip-git.pl -u {https_target}


## Dirhunt

In [ ]:
!time docker run -it --rm secsi/dirhunt --progress-disabled {target} | grep -Ev "No interesting files detected"


## dnscan

In [ ]:
%%bash -s "$subdomain_recon" "$target"

if [[ "$1" == "1" ]]; then
    time docker run -it --rm secsi/dnscan -d $2 -w subdomains-10000.txt -t 10 -R 1.0.0.1
else
    echo "Subdomain scans have been disabled."
fi


## fierce

In [1]:
!time docker run -it --rm secsi/fierce --domain {target}


NS: failure
SOA: failure
Failed to lookup NS/SOA, Domain does not exist
docker run -it --rm secsi/fierce --domain {target}  0.02s user 0.00s system 0% cpu 1:00.63 total


## Nuclei

In [ ]:
!time docker run --rm -v "`pwd`:/mnt" projectdiscovery/nuclei -u {https_target}:{port}


## TLS Scan

In [ ]:
!time testssl.sh -oA tools/testssl {target}:{port}


## Nettacker

In [ ]:
!time docker run -it --rm -v "`pwd`/tools/nettacker:/usr/src/owaspnettacker/.data/results/" nettacker python3 ./nettacker.py -m all -i {https_target}


In [ ]:
!echo "Full Nettacker results in file://`pwd`/tools/nettacker"


## Raccoon

In [ ]:
!time docker run --rm evyatarmeged/raccoon:latest {https_target} --ignored-response-codes "301,302,400,401,403,404,503,504"


## ffuf (directory brute)


In [ ]:
!echo https://{ffuf_directory_brute}
!time ffuf -s -ac -sf -of all -w /home/user/git/wordlists/SecLists/Discovery/Web-Content/raft-large-directories.txt -u https://{ffuf_directory_brute} -od tools/ffuf_directory
!echo "`find tools/ffuf_directory -type f | wc -l` responses worth investigating.."
!find tools/ffuf_directory -type f -exec grep -hE 'GET |^HTTP/' {} \; -exec echo \;


## dnsenum

In [ ]:
%%bash -s "$subdomain_recon" "$top_target"

if [[ "$1" == "1" ]]; then
    time docker run -it --rm guidelacour/dnsenum ./dnsenum.pl $2 --file dns.txt
fi


# Manual

## OpenVAS

https://localhost:8030/login/login.html (admin / admin)

## Spiderfoot

http://localhost:8060/

## Nikto

In [ ]:
!echo docker run -it --rm --net=host -v \"`pwd`/tools/nikto:/mnt/\" sullo/nikto -ask No -nointeractive -Format json -output /mnt/nikto -h {https_target}


## sn1per

Not run automatically because of large output and slow scan time

In [ ]:
!echo docker run -v \"`pwd`/tools/sn1per:/usr/share/sniper/loot/workspace/\" --rm -it xer0dayz/sn1per sniper -t {target}


In [ ]:
!echo find \"$PWD/tools/sn1per/"*"/output\" -type f -exec cat {} \\\;


# End Time

In [ ]:
!date
